In [1]:
import torch

In [2]:
import os
from glob import glob # 전체경로에서 공통적인 경로를 불러오면 리스트를 한번에 불러오는거

import numpy as np
from PIL import Image # 이미지 열 때

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import tensorflow as tf

In [4]:
# 경로 찾기
print(os.getcwd()) # 현재경로
os.listdir() # 현재 경로에있는 파일, 폴더 목록

c:\Users\kusw-04\Desktop\vsws\deep


['01.ipynb', '01real.ipynb', 'dataset']

In [5]:
train_folder = './dataset/mnist_png/training'

#os.listdir(train_folder)

In [6]:
# 파일 하나하나의 풀 경로를 받고싶음

data_paths = glob(f'{train_folder}/*/*.png')

In [7]:
len(data_paths)

60000

In [8]:
path = data_paths[0]

# 데이터 전처리

In [9]:
label_nums = os.listdir(f'{train_folder}')

## 데이터 별 개수 비교

In [10]:
nums_dataset = []

for lbl_n in label_nums:
    data_per_class = os.listdir(f'{train_folder}/{lbl_n}')
    nums_dataset.append(len(data_per_class))


## Pillow로 이미지 열기

In [12]:
image_pil = Image.open(path)
image = np.array(image_pil)

image.shape

(28, 28)

## Tensorflow로도 열수있음

In [14]:
gfile = tf.io.read_file(path)
image = tf.io.decode_image(gfile)

image.shape

TensorShape([28, 28, 1])

In [18]:
image[:, :, 0] # 이렇게 해줘야 plt.show 가능
print(1)

1


NameError: name '가능' is not defined

## Label 얻기

In [23]:
label = path.split('\\')[-2]

int(label)

0

In [24]:
def get_label(path):
    class_name = path.split('\\')[-2]
    label = int(class_name)

In [25]:
get_label(path)

## 데이터 이미지 사이즈 알기
> 데이터 사이즈가 다 달라서 사이즈를 읽고 이거에 맞춰서 개별적으로 resize를 해주는게 좋음

In [30]:
from tqdm import tqdm_notebook # 진행과정을 프로그래스바로 표현해줌

In [31]:
heights = []
widths = []

In [32]:
len(data_paths)

60000

In [33]:
for path in tqdm_notebook(data_paths): # 여기서 이렇게 tqdm_notebook을 써주면 진행과정을 볼 수 있지롱
    img_pil = Image.open(path)
    image = np.array(img_pil)
    h, w = image.shape

    heights.append(h)
    widths.append(w)

  0%|          | 0/60000 [00:00<?, ?it/s]

In [36]:
print(np.unique(heights))
print(np.unique(widths))    # 튀는 값 찾기

[28]
[28]


plt.figure(figsize=(20, 10))

---

plt.subplot(121)
plt.hist(heights)
plt.title('Heights')

plt.axvline(np.mean(heights), color = 'r', linestyle='dashed', linewidth=2) # 평균값

---

plt.subplot(122)
plt.hist(widths)
plt.title('Widths')

plt.axvline(np.mean(heights), color = 'r', linestyle='dashed', linewidth=2)

---

plt.show()

# 2차시

In [50]:
data_paths = glob('./dataset/cifar/train/*.png') # 모든 /dataset/cifar/train... 폴더를 불러서 리스트에 넣음 근데 포인트는 '전체 상대경로를 표시해준다는거!
# os.listdir 얘는 목록만 보여줌

In [53]:
path = data_paths[0]
path

'./dataset/cifar/train\\0_frog.png'

In [55]:
gfile = tf.io.read_file(path)
image = tf.io.decode_image(gfile, dtype=tf.float32)

In [57]:
image.shape

TensorShape([32, 32, 3])

In [58]:
def read_image(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile, dtype=tf.float32)
    return image

In [66]:
read_image(data_paths[0])
print(1)

1


## Batch 사이즈 설정
> 60000개의 데이터를 쪼개서 넣기

In [63]:
import cv2

In [72]:
batch_images = []

In [74]:
for path in data_paths[:8]:
    image = read_image(path)
    #image = cv2.resize(image, (32, 32)) 지금은 이미지가 안읽혀서 안되는듯
    batch_images.append(image)

In [76]:
np.array(batch_images).shape    # 사이즈가 동일(32x32)하지 않을 경우, ndarray꼴로 변경 안 묶일수도 있음

(8, 32, 32, 3)

In [77]:
batch = tf.convert_to_tensor(batch_images) # 이렇게 tensor로 묶을수도있지롱 근데 뭐 어차피 ndarray꼴로 바꿔도 tf가 tensor로 바꿔서 계산하긴함

## tensor로 만들어주는 함수

In [81]:
def make_batch(batch_paths):
    
    batch_images = []

    for path in batch_paths:
        image = read_image(path)
        batch_images.append(image)
    
    return tf.convert_to_tensor(batch_images)   # tensor화 시키기

In [82]:
batch_images = make_batch(data_paths[:8])

In [84]:
batch_images.shape

TensorShape([8, 32, 32, 3])

In [87]:
batch_size = 8
for step in range(8):
    batch_images = make_batch(data_paths[step * batch_size : (step+1) * batch_size])

    # plt.imshow(batch_images[0])
    # plt.show()

# 3차시

In [90]:
data_paths = glob('dataset/mnist_png/training/0/*png') 

In [91]:
path = data_paths[0]
path

'dataset/mnist_png/training/0\\1.png'

In [92]:
data_paths = tf.io.matching_files('./dataset/mnist_png/training/*/*.png')   # 이건 tensor 내장기능
data_paths[0]

<tf.Tensor: shape=(), dtype=string, numpy=b'.\\dataset\\mnist_png\\training\\0\\1.png'>

In [93]:
gfile = tf.io.read_file(path)

In [94]:
image = tf.io.decode_image(gfile)   # ndarray 꼴로 변경해줌

In [96]:
image.shape
# plt.imshow(image[:, :, 0], 'gray')
#plt.show()

TensorShape([28, 28, 1])

## Data Generator

In [97]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [98]:
datagen = ImageDataGenerator( #전처리할것들을 넣어주면됨 #Augmentation
    rotation_range = 20,    # 사진돌리기
    width_shift_range= 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True
)

In [100]:
image.shape

inputs = image[tf.newaxis, ...]
inputs.shape

TensorShape([1, 28, 28, 1])

In [ ]:
image = next(iter(datagen.flow(inputs)))  # Object객체에 있는 iter메소드를 이용하여 하나씩 엑세스할수있는 데이터타입으로 변환한뒤, next메소드를 이용하여 순서대로 하나씩 불러오기

```python
plt.subplot(121)
plt.imshow(np.squeeze(inputs), 'gray')
```